# <center> rQCNN

conv=2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorcircuit as tc
from jax.config import config
#import cotengra as ctg

config.update("jax_enable_x64", True)
tc.set_backend("tensorflow")
tc.set_dtype("complex128")

#opt_ctg = ctg.ReusableHyperOptimizer(
#    methods=["greedy", "kahypar"],
#    parallel="ray",
#    minimize="combo",
#    max_time=10,
#    max_repeats=128,
#    progbar=True,
#)

#tc.set_contractor("custom", optimizer=opt_ctg, preprocessing=True)

('complex128', 'float64')

## energy
$$ \hat{H}_{Ising}=J\sum_{i}{Z_{i}Z_{i+1}}-B_{x}\sum_{i}{X_{i}} $$

In [2]:
# j=1, Bx=1
def energy(c: tc.Circuit, j: float = 1.0, h: float = 1.0):
    e = 0.0
    n = c._nqubits
    for i in range(n):
        e -= h * c.expectation((tc.gates.x(), [i]))  # <X_i>
    for i in range(n - 1):  # OBC
        e += j * c.expectation(
            (tc.gates.z(), [i]), (tc.gates.z(), [i+1])
        )  # <Z_i Z_{i+1}>
    return tc.backend.real(e)

## rQCNN net

In [6]:
def rQCNN(weights):
    n = 16
    weights = tc.backend.cast(weights, "complex128")
    c = tc.Circuit(n)
    
    # pooling layer 1
    for i in range(0, n, 2):
        c.exp1(i, theta=weights[i], unitary=tc.gates._z_matrix)
    for i in range(0, n, 4):
        c.cz((i+2)%n, i)    

    # convolution layer 1
    for i in range(0, n, 2):
        c.exp1(i, (i+2)%n, theta=weights[i+8], unitary=tc.gates._xx_matrix)
        c.exp1(i, (i+2)%n, theta=weights[i+9], unitary=tc.gates._zz_matrix)    

    # pooling layer 2
    for i in range(1, n, 2):
        c.exp1(i, theta=weights[i+24], unitary=tc.gates._z_matrix)
    for i in range(0, n, 2):
        c.cz(i+1, i)  
        
    # convolution layer 2
    for i in range(n):
        c.exp1(i, (i+1)%n, theta=weights[i*2+32], unitary=tc.gates._xx_matrix)
        c.exp1(i, (i+1)%n, theta=weights[i*2+33], unitary=tc.gates._zz_matrix)

    # measure
    e = energy(c)
    return e

## train

In [8]:
vqe_tfim_vag = tc.backend.jit(
    tc.backend.vectorized_value_and_grad(rQCNN), static_argnums=(1, 2)
)

def batched_train_step_tf(batch, maxiter=10000):
    weights = tf.Variable(
        initial_value=tf.random.normal(
            shape=[batch, 64], stddev=1, dtype=getattr(tf, tc.rdtypestr)
        )
    )
    opt = tf.keras.optimizers.Adam(0.005)
    lowest_energy = 1e5
    for i in range(maxiter):
        e, grad = vqe_tfim_vag(weights)
        opt.apply_gradients([(grad, weights)])
        if tf.reduce_min(e)<lowest_energy:
            lowest_energy = tf.reduce_min(e)
        if i % 200 == 0:
            print(e)
    return lowest_energy

energy = batched_train_step_tf(10, 2000)  # batch_size, maxiter

tf.Tensor(
[ 0.97554461  1.27901607 -1.31450956  1.28051616 -0.34666035  1.00002228
 -0.48871191 -1.2838175   1.23904413 -0.095901  ], shape=(10,), dtype=float64)
tf.Tensor(
[ -8.98516732  -8.29217112 -13.95429173 -12.98397044  -8.98741656
 -10.96671463  -8.98691299 -12.91798567 -14.79714958 -10.98676173], shape=(10,), dtype=float64)
tf.Tensor(
[ -9.          -8.99976782 -14.99999973 -12.99999992  -8.99999764
 -12.99997242  -8.99997628 -12.99999271 -15.         -11.        ], shape=(10,), dtype=float64)
tf.Tensor(
[ -9.          -8.99999993 -15.         -12.99999997  -9.
 -13.          -9.         -13.         -15.         -11.        ], shape=(10,), dtype=float64)
tf.Tensor(
[ -9.          -9.         -15.         -12.99999999  -9.
 -13.          -9.         -13.         -15.         -11.        ], shape=(10,), dtype=float64)
tf.Tensor([ -9.  -9. -15. -13.  -9. -13.  -9. -13. -15. -11.], shape=(10,), dtype=float64)
tf.Tensor([ -9.  -9. -15. -13.  -9. -13.  -9. -13. -15. -11.], shape=(

## compare

In [9]:
# numerical
import quimb
h = quimb.tensor.tensor_gen.MPO_ham_ising(16, 1.0, 1.0, cyclic=False)  # Ising Hamiltonian in MPO form (number, zz interaction strenth, x-magnetic field strenth)
dmrg = quimb.tensor.tensor_dmrg.DMRG2(h, bond_dims=[10, 20, 100, 100, 200], cutoffs=1e-10)
energy0 = dmrg.solve(tol=1e-9, verbosity=1)
energy0 = dmrg.energy

#compare
print("\n")
print("numerical solution: ", energy0)
print("    rQCNN solution: ", energy.numpy())

SWEEP-1, direction=R, max_bond=(10/10), cutoff:1e-10


100%|##########################################| 15/15 [00:00<00:00, 120.37it/s]

Energy: -8.475459959874728 ... not converged.
SWEEP-2, direction=R, max_bond=(10/20), cutoff:1e-10



100%|##########################################| 15/15 [00:00<00:00, 384.61it/s]

Energy: -8.475463007473897 ... not converged.
SWEEP-3, direction=R, max_bond=(6/100), cutoff:1e-10



100%|##########################################| 15/15 [00:00<00:00, 442.93it/s]

Energy: -8.475463007532024 ... converged!


numerical solution:  -8.475463007532024
    rQCNN solution:  -15.00000000000003
